In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
training_set='F:/Birds/train'
test_set='F:/Birds/test'
validation_set= 'F:/Birds/valid'

img_size = 224

In [ ]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
   # tf.keras.layers.experimental.preprocessing.RandomFlip( mode='horizontal' ),
   # tf.keras.layers.experimental.preprocessing.RandomRotation(0.45),
   # tf.keras.layers.experimental.preprocessing.RandomWidth(0.3),
   # tf.keras.layers.experimental.preprocessing.RandomHeight(0.3),
   # tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)

])
data_aug_test = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)
])

In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(training_set, labels='inferred', label_mode='int',color_mode='rgb',batch_size=128, image_size=(img_size , img_size),shuffle=True)
num_classes = len(train.class_names)
train= train.map(lambda x,y:(data_aug(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
validation = tf.keras.preprocessing.image_dataset_from_directory(validation_set,label_mode='int',color_mode='rgb',batch_size=128,image_size=(img_size,img_size),shuffle=True,seed=52)
validation = validation.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
test = tf.keras.preprocessing.image_dataset_from_directory(test_set,label_mode='int',color_mode='rgb',batch_size=128,image_size=(img_size,img_size),shuffle=True,seed=52)
test = test.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(512,2,2,padding='same',activation='relu',input_shape=(img_size,img_size,3)),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(512,2,2,padding='same',activation='relu'),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(1536,2,2,padding='same',activation='relu'),
        #tf.keras.layers.MaxPooling2D(2),
        #tf.keras.layers.Conv2D(1024,2,2,padding='same',activation='relu'),
        # tf.keras.layers.MaxPooling2D(2),
       # tf.keras.layers.Conv2D(1024,2,2,padding='same',activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2048,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes,activation='softmax')
])


In [ ]:
model.summary()
#tf.keras.utils.plot_model(model,to_file='model.png',show_shapes=True, show_dtype=True)

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
history = model.fit(train,epochs=10,validation_data=validation)

In [ ]:
model.evaluate(test)